In [3]:
import sqlite3
import pandas as pd

# Connect to Chinook database
conn = sqlite3.connect('chinook.db')

print("Connected to Chinook database")

Connected to Chinook database


In [4]:
# Configure pandas display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', 30)

In [5]:
SCHEMA REFERENCE
Key Tables andColumns
tracks

TrackId, Name, AlbumId, MediaTypeId, GenreId, Composer
Milliseconds, Bytes, UnitPrice

albums

AlbumId, Title, ArtistId

artists

ArtistId, Name

customers

CustomerId, FirstName, LastName, Company, Address, City, State, Country, PostalCode, Phone, Fax, Email, SupportRepId

invoices

InvoiceId, CustomerId, InvoiceDate, BillingAddress, BillingCity, BillingState, BillingCountry, BillingPostalCode, Total

invoice_items

InvoiceLineId, InvoiceId, TrackId, UnitPrice, Quantity

genres

GenreId, Name

media_types

MediaTypeId, Name

employees

EmployeeId, LastName, FirstName, Title, ReportsTo, BirthDate, HireDate, Address, City, State, Country, PostalCode, Phone, Fax, Email

SyntaxError: invalid syntax (1543158634.py, line 1)